<a href="https://colab.research.google.com/github/nanafish/ORS/blob/main/%E9%9B%BB%E8%BB%8A%E4%B9%97%E3%82%8A%E6%8F%9B%E3%81%8810%E5%88%86%E3%83%9A%E3%83%8A%E3%83%AB%E3%83%86%E3%82%A3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
# ===== 1セル：0.5km内“出発駅群”→30分圏（common/union）+ 行政名付与（縮小なし＆sjoin衝突修正）=====
!pip -q install geopandas shapely pyproj fiona

import os, math, csv, unicodedata, difflib, numpy as np, pandas as pd, geopandas as gpd, logging
from shapely.geometry import Point

# ---------- 基本 ----------
os.environ["SHAPE_ENCODING"] = "CP932"
logging.getLogger("fiona").setLevel(logging.ERROR)
logging.getLogger("fiona.ogrext").setLevel(logging.ERROR)
def nrm(s): return unicodedata.normalize("NFKC", str(s)).strip()

# ---------- 入力 ----------
N02_BASE = "/content/N02-23_Station"
S12_BASE = "/content/S12-23_NumberOfPassengers"
S12_GEOJSON = "/content/S12-23_NumberOfPassengers.geojson"
N03_CANDIDATES = [
    "/content/N03-20250101_15.geojson",
    "/content/N03-20240101.geojson",
    "/content/N03-20240101_15.geojson",
    "/content/N03-20240101.gpkg",
]
NAME_FIELD_OVERRIDE_N02 = "N02_駅名"

# ---------- パラメータ（縮小なし） ----------
RADIUS_KM_FOR_MULTI_ORIGINS = 0.5
BAND_MIN = 30
BAND_SPEED_KMH = 32.0
SHRINK_RATIO   = 1.00
NEAREST_JOIN_M = 300

# ---------- 距離/時間 ----------
def hav_km(a_lat,a_lon,b_lat,b_lon):
    R=6371.0088
    A=math.radians(a_lat); B=math.radians(b_lat)
    dlat=math.radians(b_lat-a_lat); dlon=math.radians(b_lon-a_lon)
    h=math.sin(dlat/2)**2 + math.cos(A)*math.cos(B)*math.sin(dlon/2)**2
    return 2*R*math.asin(math.sqrt(h))

def band_max_km(minutes=BAND_MIN):
    return BAND_SPEED_KMH * minutes / 60.0 * SHRINK_RATIO

# ---------- N02（駅） ----------
for ext in [".shp",".shx",".dbf"]:
    if not os.path.exists(N02_BASE+ext):
        raise FileNotFoundError(f"N02が不足: {N02_BASE+ext}")

try: g = gpd.read_file(N02_BASE + ".shp")
except Exception: g = gpd.read_file(N02_BASE + ".shp", encoding="cp932")
if g.crs is None: g.set_crs(4326, inplace=True)
try: g = g.to_crs(4326)
except: pass

g = g[g.geometry.notnull() & (~g.geometry.is_empty)].copy()
if (g.geom_type == "MultiPoint").any():
    g = g.explode(index_parts=False).reset_index(drop=True)
if not (g.geom_type == "Point").all():
    W = g.to_crs(3857); W["geometry"] = W.geometry.representative_point(); g = W.to_crs(4326)

def pick_name_col(df):
    if NAME_FIELD_OVERRIDE_N02 and NAME_FIELD_OVERRIDE_N02 in df.columns: return NAME_FIELD_OVERRIDE_N02
    cand = [c for c in df.columns if ("駅" in str(c)) and pd.api.types.is_string_dtype(df[c])]
    if cand: return cand[0]
    best,score=None,-1
    for c in df.columns:
        if not pd.api.types.is_string_dtype(df[c]): continue
        s = df[c].astype(str).fillna("")
        sc = s.str.contains("駅", na=False).mean() \
             - s.str.contains(r"(線|方面|支線|ライン)$", na=False).mean()*0.9 \
             - s.str.fullmatch(r"\d+").fillna(False).mean()*0.5
        if sc>score: best,score=c,sc
    return best

def pick_line_col(df):
    for c in ["N02_路線名","路線名","路線","Line","LINE"]:
        if c in df.columns and pd.api.types.is_string_dtype(df[c]): return c
    best,score=None,-1
    for c in df.columns:
        if pd.api.types.is_string_dtype(df[c]):
            s=df[c].astype(str)
            sc=(s.str.contains("線", na=False).mean())+(s.str.len().mean()>1)*0.1
            if sc>score: best,score=c,sc
    return best

def to_lines(val):
    if not isinstance(val,str): return set()
    for z in ["／","、","・",";","；","/","|",","]: val = val.replace(z," ")
    toks=[t.strip() for t in val.split() if t.strip()]
    return set([t for t in toks if "線" in t])

name_col = pick_name_col(g)
line_col = pick_line_col(g)
if not name_col:
    raise RuntimeError(f"駅名列が特定できません。列例: {list(g.columns)[:12]} ...")

g["Name"] = g[name_col].astype(str).map(nrm)
g["LineSet"] = g[line_col].astype(str).map(to_lines) if line_col else [set()]*len(g)
g["Latitude"]  = g.geometry.y
g["Longitude"] = g.geometry.x

tmp = g[["Name","Latitude","Longitude","LineSet"]].copy()
tmp["LineSetFS"] = tmp["LineSet"].apply(lambda s: frozenset(s) if isinstance(s, set) else frozenset())
st_pts = (
    tmp.dropna(subset=["Latitude","Longitude"])
       .drop_duplicates(subset=["Name","Latitude","Longitude","LineSetFS"])
       .drop(columns=["LineSetFS"])
       .reset_index(drop=True)
)

names_unique = (
    st_pts.groupby("Name", as_index=False)[["Latitude","Longitude"]]
          .agg({"Latitude":"mean","Longitude":"mean"})
)

def choose_origin(df_names, q):
    qn=nrm(q); base=qn.replace("駅","")
    hit = df_names[df_names["Name"]==qn]
    if hit.empty: hit = df_names[df_names["Name"].str.replace("駅","",regex=False)==base]
    if not hit.empty: return hit.iloc[0]
    part = df_names[df_names["Name"].str.contains(base, na=False)]
    if part.empty:
        names = df_names["Name"].tolist()
        scored = [(difflib.SequenceMatcher(None, qn, n).ratio(), n) for n in names]
        scored.sort(reverse=True)
        cand = df_names[df_names["Name"].isin([n for _,n in scored[:20]])]
    else:
        cand = part.copy()
    print("\n候補（番号選択、Enter=1）:")
    cand = cand.assign(_sim=cand["Name"].apply(lambda n: difflib.SequenceMatcher(None, qn, n).ratio())) \
               .sort_values(["_sim","Name"], ascending=[False,True]).head(20).reset_index(drop=True)
    for i, r in cand.iterrows():
        print(f"{i+1:2d}: {r['Name']} ({r['Latitude']:.5f},{r['Longitude']:.5f})")
    sel=input("番号: ").strip(); idx = 0
    if sel.isdigit():
        v=int(sel);
        if 1<=v<=len(cand): idx=v-1
    return cand.iloc[idx]

# ---------- S12（乗降人員） ----------
def load_s12():
    old = gpd.options.io_engine; gpd.options.io_engine = "fiona"
    try:
        if os.path.exists(S12_GEOJSON):
            s = gpd.read_file(S12_GEOJSON)
        elif all(os.path.exists(S12_BASE+e) for e in [".shp",".shx",".dbf"]):
            try: s = gpd.read_file(S12_BASE + ".shp", encoding="cp932")
            except UnicodeDecodeError: s = gpd.read_file(S12_BASE + ".shp", encoding="shift_jis")
        else:
            return None
    finally:
        gpd.options.io_engine = old
    if s.crs is None: s.set_crs(4326, inplace=True)
    try: s = s.to_crs(4326)
    except: pass
    s = s[s.geometry.notnull() & (~s.geometry.is_empty)].copy()
    if (s.geom_type == "MultiPoint").any():
        s = s.explode(index_parts=False).reset_index(drop=True)
    if not (s.geom_type == "Point").all():
        W = s.to_crs(3857); W["geometry"]=W.geometry.representative_point(); s=W.to_crs(4326)
    return s

s12 = load_s12()
pass_map = {}
if s12 is not None and not s12.empty:
    name_cands = [c for c in s12.columns if ("駅" in str(c)) and pd.api.types.is_string_dtype(s12[c])] \
                 or [c for c in s12.columns if pd.api.types.is_string_dtype(s12[c])]
    pnum_cands = [c for c in s12.columns if pd.api.types.is_numeric_dtype(s12[c])]
    s12["__name__"] = s12[name_cands[0]].astype(str).map(nrm) if name_cands else ""
    best_pcol, best_score = None, -1
    for c in pnum_cands:
        ser = s12[c]; mv = ser.replace([np.inf,-np.inf], np.nan).dropna().mean()
        zr = (ser==0).mean()
        sc = float(mv)*(1.0-float(zr))
        if sc>best_score: best_pcol, best_score = c, sc
    try:
        st_g = gpd.GeoDataFrame(st_pts, geometry=gpd.points_from_xy(st_pts["Longitude"], st_pts["Latitude"]), crs=4326).to_crs(3857)
        s12m = s12.to_crs(3857)
        joined = gpd.sjoin_nearest(st_g, s12m, how="left", max_distance=NEAREST_JOIN_M, lsuffix="L", rsuffix="R")
        if best_pcol:
            tmp = joined[["Name", best_pcol]].dropna()
            pass_map.update(tmp.groupby("Name")[best_pcol].max().to_dict())
    except Exception:
        pass
    if name_cands and best_pcol:
        sdict = s12.set_index("__name__")[best_pcol].to_dict()
        for nm in st_pts["Name"].unique():
            if nm not in pass_map:
                v = sdict.get(nm) or sdict.get(nm.replace("駅",""))
                if v is not None: pass_map[nm] = v

def passengers_label(name):
    v = pass_map.get(name) or pass_map.get(name.replace("駅",""))
    if v is None: return None
    try: return f"{int(round(float(v))):,}人/日"
    except: return str(v)

# ---------- 起点 ----------
print("（例）座標OK：35.733, 139.710（池袋付近）")
q = input("出発駅名 or 'lat,lon'：").strip()

if "," in q:
    lat0, lon0 = [float(v) for v in q.split(",",1)]
    origin_name = "(座標指定)"
else:
    chosen = choose_origin(names_unique, q)
    origin_name = chosen["Name"]
    lat0, lon0 = float(chosen["Latitude"]), float(chosen["Longitude"])

print(f"\n中心: {origin_name} ({lat0:.6f}, {lon0:.6f})")

# ---------- 0.5km内の“出発駅群” ----------
start_mask = st_pts.apply(lambda r: hav_km(lat0, lon0, r["Latitude"], r["Longitude"]) <= RADIUS_KM_FOR_MULTI_ORIGINS, axis=1)
starts = st_pts[start_mask].reset_index(drop=True)
if starts.empty:
    d2 = (st_pts["Latitude"]-lat0)**2 + (st_pts["Longitude"]-lon0)**2
    starts = st_pts.loc[[int(d2.idxmin())]].reset_index(drop=True)

print("出発駅群（0.5km以内）:")
for _, row in starts.iterrows():
    print(" -", row["Name"], f"({row['Latitude']:.5f},{row['Longitude']:.5f})")

# ---------- 30分圏：各出発駅から到達可能（直線近似 + 別路線ペナルティ+10分） ----------
R30 = band_max_km(BAND_MIN)

def adj_time_min_from_any_origin(target_row) -> float:
    """
    出発駅群のいずれかから target_row へ行く時の最小“調整後時間（分）”を返す。
    - 直線ベースの所要時間
    - origin と target の LineSet が交わらなければ +10 分を加算
    """
    best = float("inf")
    t_lat, t_lon = float(target_row["Latitude"]), float(target_row["Longitude"])
    t_lines = target_row["LineSet"] if isinstance(target_row["LineSet"], set) else set()
    for _, o in starts.iterrows():
        olat, olon = float(o["Latitude"]), float(o["Longitude"])
        km = hav_km(olat, olon, t_lat, t_lon)
        base_min = km / BAND_SPEED_KMH * 60.0
        o_lines = o["LineSet"] if isinstance(o["LineSet"], set) else set()
        share = (o_lines and t_lines and (not o_lines.isdisjoint(t_lines)))
        tmin = base_min if share else (base_min + 10.0)   # ←別路線なら +10 分
        if tmin < best:
            best = tmin
    return best

# 各出発駅ごとに「到達可能インデックス」を作るのではなく、
# “調整後時間 <= 30” の駅集合を直接求める
reachable_idx = set()
for i, r in st_pts.iterrows():
    # 自駅（完全同一座標の同一レコード）はスキップ
    if ((r["Name"] in set(starts["Name"])) and
        any(abs(r["Latitude"]-o["Latitude"])<1e-10 and abs(r["Longitude"]-o["Longitude"])<1e-10 for _, o in starts.iterrows())):
        continue
    tmin = adj_time_min_from_any_origin(r)
    if tmin <= BAND_MIN:
        reachable_idx.add(i)

# “common/union”を維持したい場合：
#   - union は reachable_idx そのもの
#   - common は「全ての出発駅から tmin<=BAND_MIN になるもの」のみ
# ただし +10 分ペナルティ込みで確認
common_idx = set()
for i in reachable_idx:
    r = st_pts.iloc[i]
    ok_all = True
    for _, o in starts.iterrows():
        olat, olon = float(o["Latitude"]), float(o["Longitude"])
        km = hav_km(olat, olon, float(r["Latitude"]), float(r["Longitude"]))
        base_min = km / BAND_SPEED_KMH * 60.0
        o_lines = o["LineSet"] if isinstance(o["LineSet"], set) else set()
        r_lines = r["LineSet"] if isinstance(r["LineSet"], set) else set()
        share = (o_lines and r_lines and (not o_lines.isdisjoint(r_lines)))
        tmin = base_min if share else (base_min + 10.0)
        if tmin > BAND_MIN:
            ok_all = False; break
    if ok_all:
        common_idx.add(i)

union_idx = reachable_idx


# ---------- 行政界ロード ----------
N03_PATH = next((p for p in N03_CANDIDATES if os.path.exists(p)), None)
admin = None
if N03_PATH:
    try:
        admin = gpd.read_file(N03_PATH)
        if admin.crs is None: admin.set_crs(4326, inplace=True)
        else: admin = admin.to_crs(4326)
        admin = admin[admin.geometry.notnull() & (~admin.geometry.is_empty)].copy()
    except Exception as e:
        print("N03読込に失敗:", e)
        admin = None

# === 差し替えセル：行政名付与（index衝突を完全回避する安全版） ===
import geopandas as gpd
import pandas as pd

def _drop_indexish_cols(df: pd.DataFrame) -> pd.DataFrame:
    """sjoin/sjoin_nearest が作る or 衝突しうる列を事前に除去"""
    drop_like = {"index", "index_left", "index_right", "index_R", "index_L", "index__adm"}
    cols = [c for c in df.columns if c in drop_like or c.startswith("index_")]
    return df.drop(columns=cols, errors="ignore")

def attach_admin(gdf_pts4326: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    """
    within → 最近傍2km → 最近傍10km の順で行政名を付与。
    sjoinの 'index_*' 衝突を“事前削除 + 一意suffix”で確実に回避。
    """
    gdf = gdf_pts4326.copy()
    gdf = _drop_indexish_cols(gdf)
    gdf = gdf.rename_axis(None)              # index名も消しておく

    # admin は外側スコープの行政界 GeoDataFrame を想定
    global admin
    if admin is None or admin.empty:
        gdf["Pref"] = "不明"; gdf["City"] = "不明"
        return gdf

    adm = _drop_indexish_cols(admin.copy())
    adm = adm.rename_axis(None)

    # 列名の推定
    pref_col = next((c for c in adm.columns if str(c).startswith("N03_001")), None)
    city_col = next((c for c in adm.columns if str(c).startswith("N03_004")), None)
    if pref_col is None or city_col is None:
        str_cols = [c for c in adm.columns if pd.api.types.is_string_dtype(adm[c])]
        pref_col = pref_col or (str_cols[0] if str_cols else None)
        city_col = city_col or (str_cols[1] if len(str_cols) > 1 else pref_col)

    # 1) within
    joined = gpd.sjoin(
        gdf, adm, how="left", predicate="within",
        lsuffix="", rsuffix="_adm"          # “index_adm” を作らせない（index名はNone）
    )
    joined["Pref"] = joined.get(pref_col, None)
    joined["City"] = joined.get(city_col, None)

    # 欠損のみ対象を抽出（以後も常に衝突列を除去）
    miss = joined[joined["City"].isna()].copy()
    if not miss.empty:
        adm_3857 = adm.to_crs(3857)
        miss_3857 = _drop_indexish_cols(miss.to_crs(3857))
        nn2 = gpd.sjoin_nearest(
            miss_3857, adm_3857, how="left", max_distance=2000,
            lsuffix="", rsuffix="_adm"
        )
        if pref_col in nn2.columns: joined.loc[miss.index, "Pref"] = nn2[pref_col].values
        if city_col in nn2.columns: joined.loc[miss.index, "City"] = nn2[city_col].values

    miss2 = joined[joined["City"].isna()].copy()
    if not miss2.empty:
        adm_3857 = adm.to_crs(3857)
        miss2_3857 = _drop_indexish_cols(miss2.to_crs(3857))
        nn10 = gpd.sjoin_nearest(
            miss2_3857, adm_3857, how="left", max_distance=10000,
            lsuffix="", rsuffix="_adm"
        )
        if pref_col in nn10.columns: joined.loc[miss2.index, "Pref"] = nn10[pref_col].values
        if city_col in nn10.columns: joined.loc[miss2.index, "City"] = nn10[city_col].values

    joined["Pref"] = joined["Pref"].fillna("不明")
    joined["City"] = joined["City"].fillna("不明")
    # 最後に衝突し得る列を掃除
    joined = _drop_indexish_cols(joined)
    return joined


    # 列名推定
    pref_col = next((c for c in adm.columns if str(c).startswith("N03_001")), None)
    city_col = next((c for c in adm.columns if str(c).startswith("N03_004")), None)
    if pref_col is None or city_col is None:
        str_cols = [c for c in adm.columns if pd.api.types.is_string_dtype(adm[c])]
        pref_col = pref_col or (str_cols[0] if str_cols else None)
        city_col = city_col or (str_cols[1] if len(str_cols)>1 else pref_col)

    joined["Pref"] = joined.get(pref_col, None)
    joined["City"] = joined.get(city_col, None)

    # 2) 最近傍 2km
    miss = joined[joined["City"].isna()].copy()
    if not miss.empty:
        adm_3857 = adm.to_crs(3857)
        miss_3857 = miss.to_crs(3857)
        for c in ["index_left","index_right"]:
            if c in miss_3857.columns: miss_3857 = miss_3857.drop(columns=[c])
        nn2 = gpd.sjoin_nearest(miss_3857, adm_3857, how="left", max_distance=2000, lsuffix="L", rsuffix="R")
        if pref_col in nn2.columns: joined.loc[miss.index, "Pref"] = nn2[pref_col].values
        if city_col in nn2.columns: joined.loc[miss.index, "City"] = nn2[city_col].values

    # 3) 最近傍 10km
    miss2 = joined[joined["City"].isna()].copy()
    if not miss2.empty:
        adm_3857 = adm.to_crs(3857)
        miss2_3857 = miss2.to_crs(3857)
        for c in ["index_left","index_right"]:
            if c in miss2_3857.columns: miss2_3857 = miss2_3857.drop(columns=[c])
        nn10 = gpd.sjoin_nearest(miss2_3857, adm_3857, how="left", max_distance=10000, lsuffix="L", rsuffix="R")
        if pref_col in nn10.columns: joined.loc[miss2.index, "Pref"] = nn10[pref_col].values
        if city_col in nn10.columns: joined.loc[miss2.index, "City"] = nn10[city_col].values

    joined["Pref"] = joined["Pref"].fillna("不明")
    joined["City"] = joined["City"].fillna("不明")
    # 余分な index_* を除去
    drop_cols = [c for c in joined.columns if c.startswith("index_")]
    return joined.drop(columns=drop_cols, errors="ignore")

# ---------- 出力レコード作成（調整後時間を保存） ----------
rows = []

def min_adjusted_time_for_row(r) -> float:
    return adj_time_min_from_any_origin(r)

def add_rows(idx_set, reach_type):
    for i in sorted(idx_set):
        r = st_pts.iloc[i]
        tmin = min_adjusted_time_for_row(r)  # ← 別路線+10分込みの最短時間
        lbl = r["Name"]; ppl = passengers_label(r["Name"])
        if ppl: lbl = f"{lbl}（{ppl}）"
        rows.append({
            "Name": r["Name"],
            "Latitude": float(r["Latitude"]),
            "Longitude": float(r["Longitude"]),
            "Time_min_est": round(tmin, 1),     # 調整後時間
            "Band": f"<= {BAND_MIN}分",
            "ReachType": reach_type,            # 'common' or 'union'
            "Label": lbl
        })

# common → union の順で追加
add_rows(common_idx, "common")
add_rows(union_idx - common_idx, "union")


# ---------- CSV保存 ----------
base_name = "(座標指定)" if origin_name == "(座標指定)" else origin_name.replace("/", "_")
out = f"/content/{base_name}_multiStart_{BAND_MIN}min_admin.csv"
(gdf_out.drop(columns="geometry").to_csv(out, index=False, encoding="utf-8-sig"))

# ---------- 結果 ----------
cnt_total = len(gdf_out)
cnt_none = (gdf_out["City"]=="不明").sum()
print("\n出力:", out, f"件数:{cnt_total} / 出発点数:{len(starts)} / 0.5km内の乗換起点を全採用")
print(f"30分圏 目安半径(km): {band_max_km():.2f}（速度{BAND_SPEED_KMH}km/h×縮小{SHRINK_RATIO}）")
print(f"行政名欠損: {cnt_none}件（{100*cnt_none/max(cnt_total,1):.1f}%）  ※within→2km→10kmで補完")
print("My Maps：CSVインポート → スタイル: ReachType=カテゴリ（'common'と'union'） → ラベル: LabelFull")


/tmp/ipython-input-1176829184.py:69: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  - s.str.contains(r"(線|方面|支線|ライン)$", na=False).mean()*0.9 \
/tmp/ipython-input-1176829184.py:69: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  - s.str.contains(r"(線|方面|支線|ライン)$", na=False).mean()*0.9 \
/tmp/ipython-input-1176829184.py:69: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  - s.str.contains(r"(線|方面|支線|ライン)$", na=False).mean()*0.9 \
/tmp/ipython-input-1176829184.py:69: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  - s.str.contains(r"(線|方面|支線|ライン)$", na=False).mean()*0.9 \
/tmp/ipython-input-1176829184.py:69: UserWarning: This pattern is interprete

（例）座標OK：35.733, 139.710（池袋付近）
出発駅名 or 'lat,lon'：35.73154225476511, 139.71132989337033

中心: (座標指定) (35.731542, 139.711330)
出発駅群（0.5km以内）:
 - 池袋 (35.72749,139.71075)
 - 池袋 (35.73061,139.71034)
 - 池袋 (35.73178,139.70655)
 - 池袋 (35.72954,139.70980)
 - 池袋 (35.73152,139.71167)
 - 池袋 (35.73144,139.71197)
 - 池袋 (35.73099,139.71106)

出力: /content/(座標指定)_multiStart_30min_admin.csv 件数:704 / 出発点数:7 / 0.5km内の乗換起点を全採用
30分圏 目安半径(km): 16.00（速度32.0km/h×縮小1.0）
行政名欠損: 1件（0.1%）  ※within→2km→10kmで補完
My Maps：CSVインポート → スタイル: ReachType=カテゴリ（'common'と'union'） → ラベル: LabelFull


In [19]:
# ===== 1セル：0.5km内“出発駅群”→30分圏（common/union）+ 行政名付与 + 「別路線のみ +10分」厳密反映 =====
!pip -q install geopandas shapely pyproj fiona

import os, math, csv, unicodedata, difflib, numpy as np, pandas as pd, geopandas as gpd, logging
from shapely.geometry import Point

# ---------- 基本 ----------
os.environ["SHAPE_ENCODING"] = "CP932"
logging.getLogger("fiona").setLevel(logging.ERROR)
logging.getLogger("fiona.ogrext").setLevel(logging.ERROR)
def nrm(s): return unicodedata.normalize("NFKC", str(s)).strip()

# ---------- 入力 ----------
N02_BASE = "/content/N02-23_Station"             # .shp/.shx/.dbf
S12_BASE = "/content/S12-23_NumberOfPassengers"  # 任意
S12_GEOJSON = "/content/S12-23_NumberOfPassengers.geojson"
N03_CANDIDATES = [
    "/content/N03-20250101_15.geojson",
    "/content/N03-20240101.geojson",
    "/content/N03-20240101_15.geojson",
    "/content/N03-20240101.gpkg",
]
NAME_FIELD_OVERRIDE_N02 = "N02_駅名"

# ---------- パラメータ ----------
RADIUS_KM_FOR_MULTI_ORIGINS = 0.5  # 出発点の多重化半径
BAND_MIN = 30                      # 分
BAND_SPEED_KMH = 32.0              # 速度（直線近似）
SHRINK_RATIO   = 1.00              # ユーザー要望：縮小なし
NEAREST_JOIN_M = 300               # S12最近傍距離

# ---------- 距離/時間 ----------
def hav_km(a_lat,a_lon,b_lat,b_lon):
    R=6371.0088
    A=math.radians(a_lat); B=math.radians(b_lat)
    dlat=math.radians(b_lat-a_lat); dlon=math.radians(b_lon-a_lon)
    h=math.sin(dlat/2)**2 + math.cos(A)*math.cos(B)*math.sin(dlon/2)**2
    return 2*R*math.asin(math.sqrt(h))

def band_max_km(minutes=BAND_MIN):
    return BAND_SPEED_KMH * minutes / 60.0 * SHRINK_RATIO

# ---------- N02 読み込み & 整形 ----------
for ext in [".shp",".shx",".dbf"]:
    if not os.path.exists(N02_BASE+ext):
        raise FileNotFoundError(f"N02が不足: {N02_BASE+ext}")

try: g = gpd.read_file(N02_BASE + ".shp")
except Exception: g = gpd.read_file(N02_BASE + ".shp", encoding="cp932")
if g.crs is None: g.set_crs(4326, inplace=True)
try: g = g.to_crs(4326)
except: pass

g = g[g.geometry.notnull() & (~g.geometry.is_empty)].copy()
if (g.geom_type == "MultiPoint").any():
    g = g.explode(index_parts=False).reset_index(drop=True)
if not (g.geom_type == "Point").all():
    W = g.to_crs(3857); W["geometry"] = W.geometry.representative_point(); g = W.to_crs(4326)

def pick_name_col(df):
    if NAME_FIELD_OVERRIDE_N02 and NAME_FIELD_OVERRIDE_N02 in df.columns: return NAME_FIELD_OVERRIDE_N02
    cand = [c for c in df.columns if ("駅" in str(c)) and pd.api.types.is_string_dtype(df[c])]
    if cand: return cand[0]
    # フォールバック（駅を重視、路線語尾を減点）
    best,score=None,-1
    for c in df.columns:
        if not pd.api.types.is_string_dtype(df[c]): continue
        s=df[c].astype(str).fillna("")
        sc = s.str.contains("駅", na=False).mean() \
             - s.str.contains(r"(線|方面|支線|ライン)$", na=False).mean()*0.9 \
             - s.str.fullmatch(r"\d+").fillna(False).mean()*0.5
        if sc>score: best,score=c,sc
    return best

def pick_line_col(df):
    for c in ["N02_路線名","路線名","路線","Line","LINE"]:
        if c in df.columns and pd.api.types.is_string_dtype(df[c]): return c
    best,score=None,-1
    for c in df.columns:
        if pd.api.types.is_string_dtype(df[c]):
            s=df[c].astype(str)
            sc=(s.str.contains("線", na=False).mean())+(s.str.len().mean()>1)*0.1
            if sc>score: best,score=c,sc
    return best

def to_lines(val):
    if not isinstance(val,str): return set()
    for z in ["／","、","・",";","；","/","|",","]: val = val.replace(z," ")
    toks=[t.strip() for t in val.split() if t.strip()]
    return set([t for t in toks if "線" in t])

name_col = pick_name_col(g)
line_col = pick_line_col(g)
if not name_col:
    raise RuntimeError(f"駅名列が特定できません。列例: {list(g.columns)[:12]} ...")

g["Name"] = g[name_col].astype(str).map(nrm)
g["LineSet"] = g[line_col].astype(str).map(to_lines) if line_col else [set()]*len(g)
g["Latitude"]  = g.geometry.y
g["Longitude"] = g.geometry.x

# LineSet(set)はunhashable → frozensetで重複排除
tmp = g[["Name","Latitude","Longitude","LineSet"]].copy()
tmp["LineSetFS"] = tmp["LineSet"].apply(lambda s: frozenset(s) if isinstance(s, set) else frozenset())
st_pts = (
    tmp.dropna(subset=["Latitude","Longitude"])
       .drop_duplicates(subset=["Name","Latitude","Longitude","LineSetFS"])
       .drop(columns=["LineSetFS"])
       .reset_index(drop=True)
)

# 駅名候補（駅優先で表示）
names_unique = (
    st_pts.groupby("Name", as_index=False)[["Latitude","Longitude"]]
          .agg({"Latitude":"mean","Longitude":"mean"})
)

def choose_origin(df_names, q):
    qn=nrm(q); base=qn.replace("駅","")
    hit = df_names[df_names["Name"]==qn]
    if hit.empty: hit = df_names[df_names["Name"].str.replace("駅","",regex=False)==base]
    if not hit.empty: return hit.iloc[0]
    part = df_names[df_names["Name"].str.contains(base, na=False)]
    if part.empty:
        names = df_names["Name"].tolist()
        scored = [(difflib.SequenceMatcher(None, qn, n).ratio(), n) for n in names]
        scored.sort(reverse=True)
        cand = df_names[df_names["Name"].isin([n for _,n in scored[:20]])]
    else:
        cand = part.copy()
    print("\n候補（番号選択、Enter=1）:")
    cand = cand.assign(_sim=cand["Name"].apply(lambda n: difflib.SequenceMatcher(None, qn, n).ratio())) \
               .sort_values(["_sim","Name"], ascending=[False,True]).head(20).reset_index(drop=True)
    for i, r in cand.iterrows():
        print(f"{i+1:2d}: {r['Name']} ({r['Latitude']:.5f},{r['Longitude']:.5f})")
    sel=input("番号: ").strip(); idx = 0
    if sel.isdigit():
        v=int(sel);
        if 1<=v<=len(cand): idx=v-1
    return cand.iloc[idx]

# ---------- S12（乗降人員：ラベル用） ----------
def load_s12():
    old = gpd.options.io_engine; gpd.options.io_engine = "fiona"
    try:
        if os.path.exists(S12_GEOJSON):
            s = gpd.read_file(S12_GEOJSON)
        elif all(os.path.exists(S12_BASE+e) for e in [".shp",".shx",".dbf"]):
            try: s = gpd.read_file(S12_BASE + ".shp", encoding="cp932")
            except UnicodeDecodeError: s = gpd.read_file(S12_BASE + ".shp", encoding="shift_jis")
        else:
            return None
    finally:
        gpd.options.io_engine = old
    if s.crs is None: s.set_crs(4326, inplace=True)
    try: s = s.to_crs(4326)
    except: pass
    s = s[s.geometry.notnull() & (~s.geometry.is_empty)].copy()
    if (s.geom_type == "MultiPoint").any():
        s = s.explode(index_parts=False).reset_index(drop=True)
    if not (s.geom_type == "Point").all():
        W = s.to_crs(3857); W["geometry"]=W.geometry.representative_point(); s=W.to_crs(4326)
    return s

s12 = load_s12()
pass_map = {}
if s12 is not None and not s12.empty:
    name_cands = [c for c in s12.columns if ("駅" in str(c)) and pd.api.types.is_string_dtype(s12[c])] \
                 or [c for c in s12.columns if pd.api.types.is_string_dtype(s12[c])]
    pnum_cands = [c for c in s12.columns if pd.api.types.is_numeric_dtype(s12[c])]
    s12["__name__"] = s12[name_cands[0]].astype(str).map(nrm) if name_cands else ""
    best_pcol, best_score = None, -1
    for c in pnum_cands:
        ser = s12[c]; mv = ser.replace([np.inf,-np.inf], np.nan).dropna().mean()
        zr = (ser==0).mean()
        sc = float(mv)*(1.0-float(zr))
        if sc>best_score: best_pcol, best_score = c, sc
    try:
        st_g = gpd.GeoDataFrame(st_pts, geometry=gpd.points_from_xy(st_pts["Longitude"], st_pts["Latitude"]), crs=4326).to_crs(3857)
        s12m = s12.to_crs(3857)
        joined = gpd.sjoin_nearest(st_g, s12m, how="left", max_distance=NEAREST_JOIN_M)
        if best_pcol:
            tmp = joined[["Name", best_pcol]].dropna()
            pass_map.update(tmp.groupby("Name")[best_pcol].max().to_dict())
    except Exception:
        pass
    if name_cands and best_pcol:
        sdict = s12.set_index("__name__")[best_pcol].to_dict()
        for nm in st_pts["Name"].unique():
            if nm not in pass_map:
                v = sdict.get(nm) or sdict.get(nm.replace("駅",""))
                if v is not None: pass_map[nm] = v

def passengers_label(name):
    v = pass_map.get(name) or pass_map.get(name.replace("駅",""))
    if v is None: return None
    try: return f"{int(round(float(v))):,}人/日"
    except: return str(v)

# ---------- 行政界（N03） ----------
N03_PATH = next((p for p in N03_CANDIDATES if os.path.exists(p)), None)
admin = None
if N03_PATH:
    try:
        admin = gpd.read_file(N03_PATH)
        if admin.crs is None: admin.set_crs(4326, inplace=True)
        else: admin = admin.to_crs(4326)
        admin = admin[admin.geometry.notnull() & (~admin.geometry.is_empty)].copy()
    except Exception as e:
        print("N03読込に失敗:", e)
        admin = None

def _drop_indexish_cols(df: pd.DataFrame) -> pd.DataFrame:
    drop_like = {"index", "index_left", "index_right", "index_R", "index_L", "index__adm"}
    cols = [c for c in df.columns if c in drop_like or c.startswith("index_")]
    return df.drop(columns=cols, errors="ignore")

def attach_admin(gdf_pts4326: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    """ within → 最近傍2km → 最近傍10km。sjoinのindex衝突を完全回避。 """
    gdf = _drop_indexish_cols(gdf_pts4326.copy()).rename_axis(None)
    global admin
    if admin is None or admin.empty:
        gdf["Pref"] = "不明"; gdf["City"] = "不明"; return gdf
    adm = _drop_indexish_cols(admin.copy()).rename_axis(None)

    # 列名推定
    pref_col = next((c for c in adm.columns if str(c).startswith("N03_001")), None)
    city_col = next((c for c in adm.columns if str(c).startswith("N03_004")), None)
    if pref_col is None or city_col is None:
        str_cols = [c for c in adm.columns if pd.api.types.is_string_dtype(adm[c])]
        pref_col = pref_col or (str_cols[0] if str_cols else None)
        city_col = city_col or (str_cols[1] if len(str_cols)>1 else pref_col)

    # 1) within
    joined = gpd.sjoin(gdf, adm, how="left", predicate="within", lsuffix="", rsuffix="_adm")
    joined["Pref"] = joined.get(pref_col, None)
    joined["City"] = joined.get(city_col, None)

    # 2) 最近傍2km
    miss = joined[joined["City"].isna()].copy()
    if not miss.empty:
        adm_3857 = adm.to_crs(3857); miss_3857 = _drop_indexish_cols(miss.to_crs(3857))
        nn2 = gpd.sjoin_nearest(miss_3857, adm_3857, how="left", max_distance=2000, lsuffix="", rsuffix="_adm")
        if pref_col in nn2.columns: joined.loc[miss.index, "Pref"] = nn2[pref_col].values
        if city_col in nn2.columns: joined.loc[miss.index, "City"] = nn2[city_col].values

    # 3) 最近傍10km
    miss2 = joined[joined["City"].isna()].copy()
    if not miss2.empty:
        adm_3857 = adm.to_crs(3857); miss2_3857 = _drop_indexish_cols(miss2.to_crs(3857))
        nn10 = gpd.sjoin_nearest(miss2_3857, adm_3857, how="left", max_distance=10000, lsuffix="", rsuffix="_adm")
        if pref_col in nn10.columns: joined.loc[miss2.index, "Pref"] = nn10[pref_col].values
        if city_col in nn10.columns: joined.loc[miss2.index, "City"] = nn10[city_col].values

    joined["Pref"] = joined["Pref"].fillna("不明")
    joined["City"] = joined["City"].fillna("不明")
    return _drop_indexish_cols(joined)

# ---------- 起点入力 ----------
print("（例）座標OK：35.733, 139.710（池袋付近）")
q = input("出発駅名 or 'lat,lon'：").strip()

# 原点座標の決定
if "," in q:
    lat0, lon0 = [float(v) for v in q.split(",",1)]
    origin_name = "(座標指定)"
else:
    chosen = choose_origin(names_unique, q)
    origin_name = chosen["Name"]
    lat0, lon0 = float(chosen["Latitude"]), float(chosen["Longitude"])

print(f"\n中心: {origin_name} ({lat0:.6f}, {lon0:.6f})")

# ---------- 0.5km圏の“出発駅群” ----------
start_mask = st_pts.apply(lambda r: hav_km(lat0, lon0, r["Latitude"], r["Longitude"]) <= RADIUS_KM_FOR_MULTI_ORIGINS, axis=1)
starts = st_pts[start_mask].reset_index(drop=True)
if starts.empty:
    d2 = (st_pts["Latitude"]-lat0)**2 + (st_pts["Longitude"]-lon0)**2
    starts = st_pts.loc[[int(d2.idxmin())]].reset_index(drop=True)

print("出発駅群（0.5km以内）:")
for _, row in starts.iterrows():
    print(" -", row["Name"], f"({row['Latitude']:.5f},{row['Longitude']:.5f})  路線:", "・".join(sorted(row["LineSet"])) if row["LineSet"] else "(不明)")

# ---------- 30分圏：ペナルティの有無で2パターン ----------
R30 = band_max_km(BAND_MIN)

# 出発群の路線ユニオン（同一路線チェック用）
start_lines_union = set().union(*[ls if isinstance(ls,set) else set() for ls in starts["LineSet"]]) if len(starts)>0 else set()

def compute_rows(penalty_when_no_shared_line_min: int) -> list:
    """
    各候補駅について、任意の出発駅 o との時間
      t = (距離km / 速度) * 60 + (共有路線が1つも無いときのみ penalty)
    の最小値で30分以内か判定。'common/union' は出発駅群ごとの包含関係で色分け。
    """
    rows = []
    reach_sets = []  # originごとの到達インデックス集合（union/common判定用）
    # あらかじめ union/common 用に originごとの到達集合を作る
    for _, o in starts.iterrows():
        olat, olon = float(o["Latitude"]), float(o["Longitude"])
        reach_idx = []
        for i, r in st_pts.iterrows():
            # 同一点の自駅は除外
            if r["Name"]==o["Name"] and abs(r["Latitude"]-olat)<1e-10 and abs(r["Longitude"]-olon)<1e-10:
                continue
            km = hav_km(olat, olon, float(r["Latitude"]), float(r["Longitude"]))
            # このoriginとdestで路線共有？
            share = False
            if isinstance(o["LineSet"], set) and isinstance(r["LineSet"], set):
                share = not o["LineSet"].isdisjoint(r["LineSet"])
            # 時間の計算（ペナルティは“共有がないときだけ”加算）
            tmin = km / BAND_SPEED_KMH * 60.0 + (0 if share else penalty_when_no_shared_line_min)
            if tmin <= BAND_MIN and km <= R30*1.05:  # ほんの少し余裕
                reach_idx.append(i)
        reach_sets.append(set(reach_idx))

    union_idx = set().union(*reach_sets) if reach_sets else set()
    common_idx = set.intersection(*reach_sets) if reach_sets else set()

    def add_rows(idx_set, reach_type):
        for i in sorted(idx_set):
            r = st_pts.iloc[i]
            # 出発群の中で最短時間（※ここでもペナルティ適用ロジックを同じに）
            best_t = 1e9
            for _, o in starts.iterrows():
                km = hav_km(float(o["Latitude"]), float(o["Longitude"]), float(r["Latitude"]), float(r["Longitude"]))
                share = False
                if isinstance(o["LineSet"], set) and isinstance(r["LineSet"], set):
                    share = not o["LineSet"].isdisjoint(r["LineSet"])
                t = km / BAND_SPEED_KMH * 60.0 + (0 if share else penalty_when_no_shared_line_min)
                if t < best_t: best_t = t
            lbl = r["Name"]; ppl = passengers_label(r["Name"])
            if ppl: lbl = f"{lbl}（{ppl}）"
            rows.append({
                "Name": r["Name"],
                "Latitude": float(r["Latitude"]),
                "Longitude": float(r["Longitude"]),
                "Time_min_est": round(best_t,1),
                "Band": f"<= {BAND_MIN}分",
                "ReachType": reach_type,
                "Label": lbl
            })

    add_rows(common_idx, "common")
    add_rows(union_idx - common_idx, "union")
    return rows

# ① ペナルティなし（0分） / ② 別路線のみ +10分
rows_no_pen = compute_rows(0)
rows_pen10 = compute_rows(10)

# ---------- 行政名付与 & CSV ----------
def to_gdf(rows):
    return gpd.GeoDataFrame(
        pd.DataFrame(rows),
        geometry=gpd.points_from_xy([r["Longitude"] for r in rows], [r["Latitude"] for r in rows]),
        crs="EPSG:4326"
    )

gdf_no_pen = attach_admin(to_gdf(rows_no_pen))
gdf_pen10  = attach_admin(to_gdf(rows_pen10))

for tag, G in [("noPenalty", gdf_no_pen), ("penalty10", gdf_pen10)]:
    base_name = origin_name if origin_name == "(座標指定)" else origin_name.replace("/", "_")
    out = f"/content/{base_name}_multiStart_{BAND_MIN}min_{tag}.csv"
    (G.drop(columns="geometry")
      .assign(AddrLabel=G["Pref"].astype(str)+G["City"].astype(str),
              LabelFull=G["Label"].astype(str)+"（"+(G["Pref"].astype(str)+G["City"].astype(str))+"）")
      .to_csv(out, index=False, encoding="utf-8-sig"))
    cnt_total = len(G); cnt_none = (G["City"].fillna("不明")=="不明").sum()
    print(f"出力: {out}  件数:{cnt_total}（行政名欠損:{cnt_none}件）")

print("\n※ My Maps インポート時は 'ReachType' をカテゴリで色分け、ラベルは 'LabelFull' 推奨。")


/tmp/ipython-input-2701191511.py:70: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  - s.str.contains(r"(線|方面|支線|ライン)$", na=False).mean()*0.9 \
/tmp/ipython-input-2701191511.py:70: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  - s.str.contains(r"(線|方面|支線|ライン)$", na=False).mean()*0.9 \
/tmp/ipython-input-2701191511.py:70: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  - s.str.contains(r"(線|方面|支線|ライン)$", na=False).mean()*0.9 \
/tmp/ipython-input-2701191511.py:70: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  - s.str.contains(r"(線|方面|支線|ライン)$", na=False).mean()*0.9 \
/tmp/ipython-input-2701191511.py:70: UserWarning: This pattern is interprete

（例）座標OK：35.733, 139.710（池袋付近）
出発駅名 or 'lat,lon'：35.7324964978089, 139.7100237517183

中心: (座標指定) (35.732496, 139.710024)
出発駅群（0.5km以内）:
 - 池袋 (35.73061,139.71034)  路線: 4号線丸ノ内線
 - 池袋 (35.73178,139.70655)  路線: 13号線副都心線
 - 池袋 (35.72954,139.70980)  路線: 8号線有楽町線
 - 池袋 (35.73152,139.71167)  路線: 山手線
 - 池袋 (35.73144,139.71197)  路線: 赤羽線
 - 池袋 (35.73099,139.71106)  路線: 東上本線
出力: /content/(座標指定)_multiStart_30min_noPenalty.csv  件数:701（行政名欠損:1件）
出力: /content/(座標指定)_multiStart_30min_penalty10.csv  件数:527（行政名欠損:0件）

※ My Maps インポート時は 'ReachType' をカテゴリで色分け、ラベルは 'LabelFull' 推奨。


In [24]:
# ===== 1セル：0.5km内“出発駅群”→30分圏（common/union）+ 行政名付与
#            同一路線は高速(例55km/h)、別路線のみ +10分 を厳密反映 =====
!pip -q install geopandas shapely pyproj fiona

import os, math, csv, unicodedata, difflib, numpy as np, pandas as pd, geopandas as gpd, logging
from shapely.geometry import Point

# ---------- 基本 ----------
os.environ["SHAPE_ENCODING"] = "CP932"
logging.getLogger("fiona").setLevel(logging.ERROR)
logging.getLogger("fiona.ogrext").setLevel(logging.ERROR)
def nrm(s): return unicodedata.normalize("NFKC", str(s)).strip()

# ---------- 入力 ----------
N02_BASE = "/content/N02-23_Station"             # .shp/.shx/.dbf
S12_BASE = "/content/S12-23_NumberOfPassengers"  # 任意
S12_GEOJSON = "/content/S12-23_NumberOfPassengers.geojson"
N03_CANDIDATES = [
    "/content/N03-20250101_15.geojson",
    "/content/N03-20240101.geojson",
    "/content/N03-20240101_15.geojson",
    "/content/N03-20240101.gpkg",
]
NAME_FIELD_OVERRIDE_N02 = "N02_駅名"

# ---------- パラメータ ----------
RADIUS_KM_FOR_MULTI_ORIGINS = 0.5   # 出発点の多重化半径
BAND_MIN = 30                       # 分
BASE_SPEED_KMH = 28.0               # 乗換あり/別路線のときの基準速度（直線近似）
SAME_LINE_SPEED_KMH = 48.0          # ★同一路線のときの高速（例: 55km/h）
NEAREST_JOIN_M = 300                # S12最近傍距離[m]
# ※縮小はしません（ユーザー要望）

# ---------- 距離/時間 ----------
def hav_km(a_lat,a_lon,b_lat,b_lon):
    R=6371.0088
    A=math.radians(a_lat); B=math.radians(b_lat)
    dlat=math.radians(b_lat-a_lat); dlon=math.radians(b_lon-a_lon)
    h=math.sin(dlat/2)**2 + math.cos(A)*math.cos(B)*math.sin(dlon/2)**2
    return 2*R*math.asin(math.sqrt(h))

# ---------- N02 読み込み & 整形 ----------
for ext in [".shp",".shx",".dbf"]:
    if not os.path.exists(N02_BASE+ext):
        raise FileNotFoundError(f"N02が不足: {N02_BASE+ext}")

try: g = gpd.read_file(N02_BASE + ".shp")
except Exception: g = gpd.read_file(N02_BASE + ".shp", encoding="cp932")
if g.crs is None: g.set_crs(4326, inplace=True)
try: g = g.to_crs(4326)
except: pass

g = g[g.geometry.notnull() & (~g.geometry.is_empty)].copy()
if (g.geom_type == "MultiPoint").any():
    g = g.explode(index_parts=False).reset_index(drop=True)
if not (g.geom_type == "Point").all():
    W = g.to_crs(3857); W["geometry"] = W.geometry.representative_point(); g = W.to_crs(4326)

def pick_name_col(df):
    if NAME_FIELD_OVERRIDE_N02 and NAME_FIELD_OVERRIDE_N02 in df.columns: return NAME_FIELD_OVERRIDE_N02
    cand = [c for c in df.columns if ("駅" in str(c)) and pd.api.types.is_string_dtype(df[c])]
    if cand: return cand[0]
    best,score=None,-1
    for c in df.columns:
        if not pd.api.types.is_string_dtype(df[c]): continue
        s=df[c].astype(str).fillna("")
        sc = s.str.contains("駅", na=False).mean() \
           - s.str.contains(r"(線|方面|支線|ライン)$", na=False).mean()*0.9 \
           - s.str.fullmatch(r"\d+").fillna(False).mean()*0.5
        if sc>score: best,score=c,sc
    return best

def pick_line_col(df):
    for c in ["N02_路線名","路線名","路線","Line","LINE"]:
        if c in df.columns and pd.api.types.is_string_dtype(df[c]): return c
    best,score=None,-1
    for c in df.columns:
        if pd.api.types.is_string_dtype(df[c]):
            s=df[c].astype(str)
            sc=(s.str.contains("線", na=False).mean())+(s.str.len().mean()>1)*0.1
            if sc>score: best,score=c,sc
    return best

def to_lines(val):
    if not isinstance(val,str): return set()
    for z in ["／","、","・",";","；","/","|",","]: val = val.replace(z," ")
    toks=[t.strip() for t in val.split() if t.strip()]
    # 正規化（全角→半角、スペース除去）
    toks=[nrm(t) for t in toks]
    return set([t for t in toks if "線" in t])

name_col = pick_name_col(g)
line_col = pick_line_col(g)
if not name_col:
    raise RuntimeError(f"駅名列が特定できません。列例: {list(g.columns)[:12]} ...")

g["Name"] = g[name_col].astype(str).map(nrm)
g["LineSet"] = g[line_col].astype(str).map(to_lines) if line_col else [set()]*len(g)
g["Latitude"]  = g.geometry.y
g["Longitude"] = g.geometry.x

# set は unhashable → frozenset で重複排除
tmp = g[["Name","Latitude","Longitude","LineSet"]].copy()
tmp["LineSetFS"] = tmp["LineSet"].apply(lambda s: frozenset(s) if isinstance(s, set) else frozenset())
st_pts = (
    tmp.dropna(subset=["Latitude","Longitude"])
       .drop_duplicates(subset=["Name","Latitude","Longitude","LineSetFS"])
       .drop(columns=["LineSetFS"])
       .reset_index(drop=True)
)

# 駅名候補（駅優先で表示）
names_unique = (
    st_pts.groupby("Name", as_index=False)[["Latitude","Longitude"]]
          .agg({"Latitude":"mean","Longitude":"mean"})
)

def choose_origin(df_names, q):
    qn=nrm(q); base=qn.replace("駅","")
    hit = df_names[df_names["Name"]==qn]
    if hit.empty: hit = df_names[df_names["Name"].str.replace("駅","",regex=False)==base]
    if not hit.empty: return hit.iloc[0]
    part = df_names[df_names["Name"].str.contains(base, na=False)]
    if part.empty:
        names = df_names["Name"].tolist()
        scored = [(difflib.SequenceMatcher(None, qn, n).ratio(), n) for n in names]
        scored.sort(reverse=True)
        cand = df_names[df_names["Name"].isin([n for _,n in scored[:20]])]
    else:
        cand = part.copy()
    print("\n候補（番号選択、Enter=1）:")
    cand = cand.assign(_sim=cand["Name"].apply(lambda n: difflib.SequenceMatcher(None, qn, n).ratio())) \
               .sort_values(["_sim","Name"], ascending=[False,True]).head(20).reset_index(drop=True)
    for i, r in cand.iterrows():
        print(f"{i+1:2d}: {r['Name']} ({r['Latitude']:.5f},{r['Longitude']:.5f})")
    sel=input("番号: ").strip(); idx = 0
    if sel.isdigit():
        v=int(sel);
        if 1<=v<=len(cand): idx=v-1
    return cand.iloc[idx]

# ---------- S12（乗降人員：ラベル用／任意） ----------
def load_s12():
    old = gpd.options.io_engine; gpd.options.io_engine = "fiona"
    try:
        if os.path.exists(S12_GEOJSON):
            s = gpd.read_file(S12_GEOJSON)
        elif all(os.path.exists(S12_BASE+e) for e in [".shp",".shx",".dbf"]):
            try: s = gpd.read_file(S12_BASE + ".shp", encoding="cp932")
            except UnicodeDecodeError: s = gpd.read_file(S12_BASE + ".shp", encoding="shift_jis")
        else:
            return None
    finally:
        gpd.options.io_engine = old
    if s.crs is None: s.set_crs(4326, inplace=True)
    try: s = s.to_crs(4326)
    except: pass
    s = s[s.geometry.notnull() & (~s.geometry.is_empty)].copy()
    if (s.geom_type == "MultiPoint").any():
        s = s.explode(index_parts=False).reset_index(drop=True)
    if not (s.geom_type == "Point").all():
        W = s.to_crs(3857); W["geometry"]=W.geometry.representative_point(); s=W.to_crs(4326)
    return s

s12 = load_s12()
pass_map = {}
if s12 is not None and not s12.empty:
    name_cands = [c for c in s12.columns if ("駅" in str(c)) and pd.api.types.is_string_dtype(s12[c])] \
                 or [c for c in s12.columns if pd.api.types.is_string_dtype(s12[c])]
    pnum_cands = [c for c in s12.columns if pd.api.types.is_numeric_dtype(s12[c])]
    s12["__name__"] = s12[name_cands[0]].astype(str).map(nrm) if name_cands else ""
    best_pcol, best_score = None, -1
    for c in pnum_cands:
        ser = s12[c]; mv = ser.replace([np.inf,-np.inf], np.nan).dropna().mean()
        zr = (ser==0).mean()
        sc = float(mv)*(1.0-float(zr))
        if sc>best_score: best_pcol, best_score = c, sc
    try:
        st_g = gpd.GeoDataFrame(st_pts, geometry=gpd.points_from_xy(st_pts["Longitude"], st_pts["Latitude"]), crs=4326).to_crs(3857)
        s12m = s12.to_crs(3857)
        joined = gpd.sjoin_nearest(st_g, s12m, how="left", max_distance=NEAREST_JOIN_M)
        if best_pcol:
            tmp = joined[["Name", best_pcol]].dropna()
            pass_map.update(tmp.groupby("Name")[best_pcol].max().to_dict())
    except Exception:
        pass
    if name_cands and best_pcol:
        sdict = s12.set_index("__name__")[best_pcol].to_dict()
        for nm in st_pts["Name"].unique():
            if nm not in pass_map:
                v = sdict.get(nm) or sdict.get(nm.replace("駅",""))
                if v is not None: pass_map[nm] = v

def passengers_label(name):
    v = pass_map.get(name) or pass_map.get(name.replace("駅",""))
    if v is None: return None
    try: return f"{int(round(float(v))):,}人/日"
    except: return str(v)

# ---------- 行政界（N03） ----------
N03_PATH = next((p for p in N03_CANDIDATES if os.path.exists(p)), None)
admin = None
if N03_PATH:
    try:
        admin = gpd.read_file(N03_PATH)
        if admin.crs is None: admin.set_crs(4326, inplace=True)
        else: admin = admin.to_crs(4326)
        admin = admin[admin.geometry.notnull() & (~admin.geometry.is_empty)].copy()
    except Exception as e:
        print("N03読込に失敗:", e)
        admin = None

def _drop_indexish_cols(df: pd.DataFrame) -> pd.DataFrame:
    drop_like = {"index", "index_left", "index_right", "index_R", "index_L", "index__adm"}
    cols = [c for c in df.columns if c in drop_like or c.startswith("index_")]
    return df.drop(columns=cols, errors="ignore")

def attach_admin(gdf_pts4326: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    gdf = _drop_indexish_cols(gdf_pts4326.copy()).rename_axis(None)
    global admin
    if admin is None or admin.empty:
        gdf["Pref"] = "不明"; gdf["City"] = "不明"; return gdf
    adm = _drop_indexish_cols(admin.copy()).rename_axis(None)
    pref_col = next((c for c in adm.columns if str(c).startswith("N03_001")), None)
    city_col = next((c for c in adm.columns if str(c).startswith("N03_004")), None)
    if pref_col is None or city_col is None:
        str_cols = [c for c in adm.columns if pd.api.types.is_string_dtype(adm[c])]
        pref_col = pref_col or (str_cols[0] if str_cols else None)
        city_col = city_col or (str_cols[1] if len(str_cols)>1 else pref_col)
    joined = gpd.sjoin(gdf, adm, how="left", predicate="within", lsuffix="", rsuffix="_adm")
    joined["Pref"] = joined.get(pref_col, None)
    joined["City"] = joined.get(city_col, None)
    miss = joined[joined["City"].isna()].copy()
    if not miss.empty:
        adm_3857 = adm.to_crs(3857); miss_3857 = _drop_indexish_cols(miss.to_crs(3857))
        nn2 = gpd.sjoin_nearest(miss_3857, adm_3857, how="left", max_distance=2000, lsuffix="", rsuffix="_adm")
        if pref_col in nn2.columns: joined.loc[miss.index, "Pref"] = nn2[pref_col].values
        if city_col in nn2.columns: joined.loc[miss.index, "City"] = nn2[city_col].values
    miss2 = joined[joined["City"].isna()].copy()
    if not miss2.empty:
        adm_3857 = adm.to_crs(3857); miss2_3857 = _drop_indexish_cols(miss2.to_crs(3857))
        nn10 = gpd.sjoin_nearest(miss2_3857, adm_3857, how="left", max_distance=10000, lsuffix="", rsuffix="_adm")
        if pref_col in nn10.columns: joined.loc[miss2.index, "Pref"] = nn10[pref_col].values
        if city_col in nn10.columns: joined.loc[miss2.index, "City"] = nn10[city_col].values
    joined["Pref"] = joined["Pref"].fillna("不明")
    joined["City"] = joined["City"].fillna("不明")
    return _drop_indexish_cols(joined)

# ---------- 起点入力 ----------
print("（例）座標OK：35.733, 139.710（池袋付近）")
q = input("出発駅名 or 'lat,lon'：").strip()

# 原点座標の決定
if "," in q:
    lat0, lon0 = [float(v) for v in q.split(",",1)]
    origin_name = "(座標指定)"
else:
    chosen = choose_origin(names_unique, q)
    origin_name = chosen["Name"]
    lat0, lon0 = float(chosen["Latitude"]), float(chosen["Longitude"])
print(f"\n中心: {origin_name} ({lat0:.6f}, {lon0:.6f})")

# ---------- 0.5km圏の“出発駅群” ----------
start_mask = st_pts.apply(lambda r: hav_km(lat0, lon0, r["Latitude"], r["Longitude"]) <= RADIUS_KM_FOR_MULTI_ORIGINS, axis=1)
starts = st_pts[start_mask].reset_index(drop=True)
if starts.empty:
    d2 = (st_pts["Latitude"]-lat0)**2 + (st_pts["Longitude"]-lon0)**2
    starts = st_pts.loc[[int(d2.idxmin())]].reset_index(drop=True)

print("出発駅群（0.5km以内）:")
for _, row in starts.iterrows():
    print(" -", row["Name"], f"({row['Latitude']:.5f},{row['Longitude']:.5f})  路線:", "・".join(sorted(row["LineSet"])) if row["LineSet"] else "(不明)")

# ---------- 30分圏：ペナルティの有無で2パターン ----------
# 出発群の路線ユニオン（情報確認用。判定自体は起点ごとに最短時間を取る）
start_lines_union = set().union(*[ls if isinstance(ls,set) else set() for ls in starts["LineSet"]]) if len(starts)>0 else set()

def compute_rows(penalty_when_no_shared_line_min: int) -> list:
    """
    各候補駅 r について、起点 o ごとに
      share_o = (oのLineSet ∩ rのLineSet ≠ ∅)
      speed   = SAME_LINE_SPEED_KMH if share_o else BASE_SPEED_KMH
      t_o     = 距離/速度*60 + (0 if share_o else penalty)
    の最小 t_o を採用し、t_o ≤ 30分 なら採用。
    """
    rows = []
    reach_sets = []
    # 起点ごとの到達集合（union/common判定用）
    for _, o in starts.iterrows():
        olat, olon = float(o["Latitude"]), float(o["Longitude"])
        reach_idx = []
        for i, r in st_pts.iterrows():
            # 自駅は除外
            if r["Name"]==o["Name"] and abs(r["Latitude"]-olat)<1e-10 and abs(r["Longitude"]-olon)<1e-10:
                continue
            km = hav_km(olat, olon, float(r["Latitude"]), float(r["Longitude"]))
            share = False
            if isinstance(o["LineSet"], set) and isinstance(r["LineSet"], set):
                share = not o["LineSet"].isdisjoint(r["LineSet"])
            speed = SAME_LINE_SPEED_KMH if share else BASE_SPEED_KMH
            tmin  = km / speed * 60.0 + (0 if share else penalty_when_no_shared_line_min)
            if tmin <= BAND_MIN:
                reach_idx.append(i)
        reach_sets.append(set(reach_idx))
    union_idx = set().union(*reach_sets) if reach_sets else set()
    common_idx = set.intersection(*reach_sets) if reach_sets else set()

    def add_rows(idx_set, reach_type):
        for i in sorted(idx_set):
            r = st_pts.iloc[i]
            # 出発群の中での最短時間（←ここでも同ロジックで厳密評価）
            best_t = 1e9
            for _, o in starts.iterrows():
                km = hav_km(float(o["Latitude"]), float(o["Longitude"]), float(r["Latitude"]), float(r["Longitude"]))
                share = False
                if isinstance(o["LineSet"], set) and isinstance(r["LineSet"], set):
                    share = not o["LineSet"].isdisjoint(r["LineSet"])
                speed = SAME_LINE_SPEED_KMH if share else BASE_SPEED_KMH
                t = km / speed * 60.0 + (0 if share else penalty_when_no_shared_line_min)
                if t < best_t: best_t = t
            lbl = r["Name"]; ppl = passengers_label(r["Name"])
            if ppl: lbl = f"{lbl}（{ppl}）"
            rows.append({
                "Name": r["Name"],
                "Latitude": float(r["Latitude"]),
                "Longitude": float(r["Longitude"]),
                "Time_min_est": round(best_t,1),
                "Band": f"<= {BAND_MIN}分",
                "ReachType": reach_type,
                "Label": lbl
            })
    add_rows(common_idx, "common")
    add_rows(union_idx - common_idx, "union")
    return rows

# ① ペナルティなし / ② 別路線のみ +10分
rows_no_pen = compute_rows(0)
rows_pen10  = compute_rows(10)

# ---------- 行政名付与 & CSV ----------
def to_gdf(rows):
    return gpd.GeoDataFrame(
        pd.DataFrame(rows),
        geometry=gpd.points_from_xy([r["Longitude"] for r in rows], [r["Latitude"] for r in rows]),
        crs="EPSG:4326"
    )

# 行政名付与
gdf_no_pen = attach_admin(to_gdf(rows_no_pen))
gdf_pen10  = attach_admin(to_gdf(rows_pen10))

# 住所ラベル付与 & 出力
for tag, G in [("noPenalty", gdf_no_pen), ("penalty10", gdf_pen10)]:
    base_name = origin_name if origin_name == "(座標指定)" else origin_name.replace("/", "_")
    out = f"/content/{base_name}_multiStart_{BAND_MIN}min_{tag}.csv"
    (G.drop(columns="geometry")
      .assign(AddrLabel=G["Pref"].astype(str)+G["City"].astype(str),
              LabelFull=G["Label"].astype(str)+"（"+(G["Pref"].astype(str)+G["City"].astype(str))+"）")
      .to_csv(out, index=False, encoding="utf-8-sig"))
    cnt_total = len(G); cnt_none = (G["City"].fillna("不明")=="不明").sum()
    print(f"出力: {out}  件数:{cnt_total}（行政名欠損:{cnt_none}件）")

print("\n※ My Maps: 'ReachType' をカテゴリで色分け、ラベルは 'LabelFull' 推奨。")
print(f"速度設定: 同一路線 {SAME_LINE_SPEED_KMH} km/h / 別路線 {BASE_SPEED_KMH} km/h、別路線のみ +10分（2本目CSV）")


/tmp/ipython-input-3876585123.py:68: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  - s.str.contains(r"(線|方面|支線|ライン)$", na=False).mean()*0.9 \
/tmp/ipython-input-3876585123.py:68: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  - s.str.contains(r"(線|方面|支線|ライン)$", na=False).mean()*0.9 \
/tmp/ipython-input-3876585123.py:68: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  - s.str.contains(r"(線|方面|支線|ライン)$", na=False).mean()*0.9 \
/tmp/ipython-input-3876585123.py:68: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  - s.str.contains(r"(線|方面|支線|ライン)$", na=False).mean()*0.9 \
/tmp/ipython-input-3876585123.py:68: UserWarning: This pattern is interprete

（例）座標OK：35.733, 139.710（池袋付近）


KeyboardInterrupt: Interrupted by user